In [1]:
import numpy as np
import torch
import math
from torch import nn
import torch.nn.functional as F
from tokenizer import Tokenizer

import regex as re
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader
from enums import CLS_ID, PAD_ID, IGNORE_INDEX, MAX_LEN, DROPOUT

# Sentence Transformer Multi-Task Expansion Overview

## Dataset
* MASSIVE dataset (English split) for multi-task learning, which contains User utterances paired with:

> Intent labels (classification task)

> Slot annotations for Named Entity Recognition (NER) (sequence tagging task)


* Key steps:

> BPE tokenization is performed using a custom tokenizer.

> Slot labels are aligned with BPE pieces, expanding one label across subword units.

> Both input IDs and slot labels are padded/truncated to a maximum length (MAX_LEN).
 

* Oversampling is used during training to give more weight to examples containing at least one NER tag (to handle class imbalance).



## Multi-Task Model Architecture

* The multitask model builds on top of the SentenceTransformer backbone:

* Intent Classifier

> A simple Linear layer applied on the [CLS] token embedding for intent classification.

* NER Tagger

> A 2-layer bidirectional LSTM applied on the output sequence embeddings.

> A 2-layer MLP (LayerNorm → Linear → GELU → Dropout → Linear) classifies each token position.

## Design choice
Initially, only the new heads were trained. After multiple unfreezing backbone experiments, unfreezing the full backbone gave better performance.

## Loss Function

* Intent Loss

> Standard CrossEntropyLoss on the intent classification output.

* NER Loss

> CrossEntropyLoss applied token-wise, with class weighting to emphasize rare NER labels.

> The loss ignores padding and CLS tokens (using IGNORE_INDEX).

#### The final loss is the sum of intent loss and NER loss.

## Training phase details:

> Weighted Random Sampler to balance NER-positive examples.

> Mixed Precision Training (AMP) for faster and memory-efficient training.

> Gradient clipping for stability.

> Cosine Annealing LR scheduler.

> Early stopping based on validation loss.

## Inference
* During inference:

> Input sentences are tokenized and passed through the multitask model.

> The model predicts: Intent label (single prediction). And NER tags for each token position (sequence prediction).

* A simple sanity check shows:

> Intent classification performs reasonably well early on.

> NER tagging performance could improve with larger models, more data, or task-specific pretraining.



In [2]:
class MultiTaskDataset(Dataset):
    def __init__(self, tokenizer_path, split):
        self.tokenizer = Tokenizer(tokenizer_path) # load tokenizer
        self.pat = self.tokenizer.pat # get compiled regex pattern
        self.IGNORE_INDEX = IGNORE_INDEX # used for values with ' ' or PAD or CLS tokens.
        self.CLS_ID = CLS_ID
        self.PAD_ID = PAD_ID
        self.dataset = load_dataset("qanastek/MASSIVE", "en-US", split=split) # Load the english NER+IntentClassification dataset based on split


    def __len__(self):
        return len(self.dataset)


    def __getitem__(self, idx):
        ex = self.dataset[idx]
        
        text  = ex["utt"]
        slots = ex["ner_tags"]
        intent = ex["intent"]
        
        word_spans = [(m.group(), m.span()) for m in re.finditer(r"\S+", text)]        
        pieces_with_spans = [(m.group(), m.span()) for m in re.finditer(self.pat, text)]
        
        piece_labels = []
        for piece, (start, end) in pieces_with_spans:
            for idx, (_, (w0, w1)) in enumerate(word_spans):
                if w0 <= start < w1:
                    piece_labels.append(slots[idx])
                    break
            else:
                piece_labels.append(self.IGNORE_INDEX)
        
        # Expand to BPE tokens
        token_pieces = self.tokenizer.encode(text)
        flat_ids     = [self.CLS_ID]
        flat_labels  = [self.IGNORE_INDEX]
        
        for tok_ids, lab in zip(token_pieces, piece_labels):
            flat_ids.extend(tok_ids)
            flat_labels.extend([lab] * len(tok_ids))
        

        return [flat_ids, flat_labels, intent]


def collate(batch):
    flat_ids, flat_labels, intents = zip(*batch)

    # Compute the joint max length (capped by MAX_LEN)
    raw_max = max(max(len(ids), len(labs)) for ids, labs in zip(flat_ids, flat_labels))
    L = min(raw_max, MAX_LEN)

    # Pad or truncate *both* ids and labels to length L
    input_ids = []
    slot_labels = []
    attention_mask = []
    for ids, labs in zip(flat_ids, flat_labels):
        ids = ids[:L]
        labs = labs[:L]

        pad_len = L - len(ids)
        input_ids.append(ids + [PAD_ID] * pad_len)
        slot_labels.append(labs + [IGNORE_INDEX] * pad_len)
        attention_mask.append([1]*len(ids) + [0]*pad_len)

    # Stack to form tensors
    input_ids = torch.tensor(input_ids, dtype=torch.long)
    attention_mask = torch.tensor(attention_mask, dtype=torch.long)
    slot_labels = torch.tensor(slot_labels, dtype=torch.long)
    intent_labels = torch.tensor(intents, dtype=torch.long)

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "slot_labels": slot_labels,
        "intent_labels": intent_labels,
    }


In [3]:
train_ds = MultiTaskDataset("bpe_merged.json", "train")
validation_ds = MultiTaskDataset("bpe_merged.json", "validation")
test_ds = MultiTaskDataset("bpe_merged.json", "test")

In [4]:
# Oversampling data with ner tags because they are under-represneted in the training dataset.
from torch.utils.data import WeightedRandomSampler

pos_indicator = []
for flat_ids, flat_labels, intent in train_ds:
    has_positive = any((lab != IGNORE_INDEX and lab != 0) for lab in flat_labels)
    pos_indicator.append(1 if has_positive else 0)

pos_alpha = 5.0
neg_alpha = 3.0

weights = [pos_alpha if p==1 else neg_alpha for p in pos_indicator]

sampler = WeightedRandomSampler(
    weights,
    num_samples=len(weights),
    replacement=True
)

In [5]:
# Finding unique number of intents and ner tags.
num_intents = len(set(train_ds.dataset["intent"] + validation_ds.dataset["intent"] + test_ds.dataset["intent"]))
num_ner_tags = train_ds.dataset.features["ner_tags"].feature.num_classes + 1

In [6]:
train_loader = DataLoader(
    train_ds,
    batch_size=32,
    sampler=sampler,
    num_workers=0,
    pin_memory=True,
    collate_fn=collate
)
val_loader = DataLoader(
    validation_ds,
    batch_size=32, shuffle=False,
    num_workers=0, pin_memory=True,
    collate_fn=collate
)
test_loader = DataLoader(
    test_ds,
    batch_size=32, shuffle=False,
    num_workers=0, pin_memory=True,
    collate_fn=collate
)

In [7]:
class MultiTaskModel(nn.Module):
    """
    Backbone = pre‐trained SentenceTransformer, plus:
     - intent model: simple linear on [CLS]
     - NER model: 2×BiLSTM → 2‐layer MLP on each timestep
    """
    def __init__(
        self,
        sentence_encoder: nn.Module,
        d_model: int,
        proj_dim: int,
        num_intents: int,
        num_ner_tags: int,
        dropout: float = DROPOUT,
        freeze_encoder: bool = True,
    ):
        super().__init__()
        self.encoder = sentence_encoder

        # intent head
        self.intent_classifier = nn.Linear(proj_dim, num_intents)

        # NER head
        self.ner_lstm = nn.LSTM(
            input_size=d_model,
            hidden_size=d_model//2,
            num_layers=2,
            bidirectional=True,
            batch_first=True,
            dropout=dropout if 2 > 1 else 0.0
        )
        self.ner_mlp = nn.Sequential(
            nn.LayerNorm(d_model),
            nn.Linear(d_model, d_model),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(d_model, num_ner_tags)
        )

    def forward(self, input_ids, attention_mask):
        # encode
        cls_emb, seq_emb = self.encoder(input_ids, attention_mask=attention_mask, return_all=True)
        intent_logits = self.intent_classifier(cls_emb)
        lstm_out, _ = self.ner_lstm(seq_emb)
        ner_logits  = self.ner_mlp(lstm_out)
        return intent_logits, ner_logits


In [8]:
from sent_transformer import SentenceTransformer
from enums import VOCAB_SIZE, D_MODEL, NHEAD, NUM_LAYERS, PROJ_DIM, DROPOUT

# Load PreTrained Sentence Encoder.
encoder = SentenceTransformer(
            vocab_size=VOCAB_SIZE, d_model=D_MODEL, nhead=NHEAD, num_layers=NUM_LAYERS, proj_dim=PROJ_DIM, dropout=DROPOUT
        )

encoder.load_state_dict(torch.load('best_encoder.pt'))

<All keys matched successfully>

In [9]:
import os
from tqdm import tqdm
import torch
from torch.optim import AdamW

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize MultiTaskModel
model = MultiTaskModel(
    sentence_encoder=encoder,
    d_model=D_MODEL,
    proj_dim=PROJ_DIM,
    num_intents=num_intents,
    num_ner_tags=num_ner_tags,
    freeze_encoder=True
).to(device)


# I tried various combinations of layer freezing. ie: freezing the whole backbone, 
# unfreezing the last 2 layers of the backbone, unfreezing the last 2 layers of the backbone along with some encoder layers
# and unfreezing the whole backbone.

# I ended up unfreezing the whole backbone for eval accuracy.

encoder_params = []
for n,p in model.encoder.named_parameters():
    if p.requires_grad:
        encoder_params.append(p)
        
head_params = (
    list(model.intent_classifier.parameters()) +
    list(model.ner_lstm.parameters()) +
    list(model.ner_mlp.parameters())
)


opt = AdamW([
    {'params': encoder_params, 'lr': 5e-5},   # fine‐tune top transformer layers with a really low LR.
    {'params': head_params, 'lr': 1e-3},   # train new heads from scratch with high LR for faster convergence.
], weight_decay=1e-2)


epochs = 500
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=epochs)

# AMP setup
use_amp = torch.cuda.is_available()
scaler = torch.cuda.amp.GradScaler() if use_amp else None

# Checkpointing and early stopping setup
checkpoint_dir = "checkpoints_multitask"
os.makedirs(checkpoint_dir, exist_ok=True)

best_val_loss = float('inf')
patience = 5
no_improve = 0

intent_loss_fn = nn.CrossEntropyLoss()

weights = torch.ones(num_ner_tags)
weights *= 3
weights[[0,-1]] = 1
ner_loss_fn = nn.CrossEntropyLoss(weight=weights, ignore_index=IGNORE_INDEX)


# Training
for epoch in range(1, epochs + 1):
    model.train()
    train_loss = 0.0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch} Train"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        slot_labels = batch["slot_labels"].to(device)
        intent_labels = batch["intent_labels"].to(device)

        opt.zero_grad()

        if use_amp:
            with torch.autocast(device_type='cuda', dtype=torch.bfloat16):
                intent_logits, ner_logits = model(input_ids, attention_mask)
                intent_loss = intent_loss_fn(intent_logits, intent_labels)
                ner_loss = ner_loss_fn(ner_logits.view(-1, num_ner_tags), slot_labels.view(-1))
                loss = intent_loss + ner_loss

            scaler.scale(loss).backward()
            scaler.unscale_(opt)
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            scaler.step(opt)
            scaler.update()
        else:
            intent_logits, ner_logits = model(input_ids, attention_mask)
            intent_loss = intent_loss_fn(intent_logits, intent_labels)
            ner_loss = ner_loss_fn(ner_logits.view(-1, num_ner_tags), slot_labels.view(-1))
            loss = intent_loss + ner_loss

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            opt.step()

        train_loss += loss.item()

    train_loss /= len(train_loader)

    # Validation
    model.eval()
    val_loss = 0.0

    with torch.no_grad():
        for batch in tqdm(val_loader, desc=f"Epoch {epoch} Val"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            slot_labels = batch["slot_labels"].to(device)
            intent_labels = batch["intent_labels"].to(device)

            if use_amp:
                with torch.autocast(device_type='cuda', dtype=torch.bfloat16):
                    intent_logits, ner_logits = model(input_ids, attention_mask)
                    intent_loss = intent_loss_fn(intent_logits, intent_labels)
                    ner_loss = ner_loss_fn(ner_logits.view(-1, num_ner_tags), slot_labels.view(-1))
                    loss = intent_loss + ner_loss
            else:
                intent_logits, ner_logits = model(input_ids, attention_mask)
                intent_loss = intent_loss_fn(intent_logits, intent_labels)
                ner_loss = ner_loss_fn(ner_logits.view(-1, num_ner_tags), slot_labels.view(-1))
                loss = intent_loss + ner_loss

            val_loss += loss.item()

    val_loss /= len(val_loader)

    # Learning Rate Schedule
    scheduler.step()
    lr = scheduler.get_last_lr()[0]

    print(f"Epoch {epoch}/{epochs} — "
          f"train_loss={train_loss:.5f}  val_loss={val_loss:.5f}  lr={lr:.1e}")

    # Saving Checkpoints
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': opt.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'train_loss': train_loss,
        'val_loss': val_loss,
    }, os.path.join(checkpoint_dir, f"multitask_epoch_{epoch}.pt"))

    # Early Stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        no_improve = 0
        torch.save(model.state_dict(), "best_multitask_model.pt")
        print("  ↳ New best MultiTaskModel saved.")
    else:
        no_improve += 1
        print(f"  ↳ No improvement for {no_improve} epoch(s).")
        if no_improve >= patience:
            print(f"Stopping early after {patience} epochs without improvement.")
            break


Epoch 1 Val:   0%|                                       | 0/64 [00:00<?, ?it/s]/Users/srinathramalingam/miniconda3/envs/sent-transform/lib/python3.10/site-packages/torch/nn/modules/transformer.py:508: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. We recommend specifying layout=torch.jagged when constructing a nested tensor, as this layout receives active development, has better operator coverage, and works with torch.compile. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/NestedTensorImpl.cpp:182.)
  output = torch._nested_tensor_from_mask(
Epoch 1 Val: 100%|██████████████████████████████| 64/64 [00:02<00:00, 29.82it/s]


Epoch 1/500 — train_loss=6.47528  val_loss=5.66544  lr=5.0e-05
  ↳ New best MultiTaskModel saved.


Epoch 2 Val: 100%|██████████████████████████████| 64/64 [00:02<00:00, 29.09it/s]


Epoch 2/500 — train_loss=5.44485  val_loss=5.11691  lr=5.0e-05
  ↳ New best MultiTaskModel saved.


Epoch 3 Val: 100%|██████████████████████████████| 64/64 [00:02<00:00, 27.75it/s]


...

Epoch 48/500 — train_loss=2.56847  val_loss=3.64161  lr=4.9e-05
  ↳ No improvement for 3 epoch(s).


Epoch 49 Val: 100%|█████████████████████████████| 64/64 [00:02<00:00, 21.48it/s]


Epoch 49/500 — train_loss=2.53413  val_loss=3.64725  lr=4.9e-05
  ↳ No improvement for 4 epoch(s).


Epoch 50 Val: 100%|█████████████████████████████| 64/64 [00:02<00:00, 23.56it/s]

Epoch 50/500 — train_loss=2.49843  val_loss=3.62813  lr=4.9e-05
  ↳ No improvement for 5 epoch(s).
Stopping early after 5 epochs without improvement.


In [10]:
# Sanity Check on a few samples on the test dataset
# Upon a quick check, intent classification performs reasonably well.
# And the NER model needs further training of the backbone on a larger architecture and dataset. 
# But this is a good first step as we see some okay predictions.

for idx in np.random.randint(1,100,10):
    batch = collate([test_ds[int(idx)]])
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    slot_labels = batch["slot_labels"].to(device)
    intent_labels = batch["intent_labels"].to(device)


    with torch.no_grad():
        intent_logits, ner_logits = model(input_ids, attention_mask)
        intent_loss = intent_loss_fn(intent_logits, intent_labels)
        ner_loss = ner_loss_fn(ner_logits.view(-1, num_ner_tags), slot_labels.view(-1))

        intent_preds = int(np.argmax(intent_logits))
        intent_labels = intent_labels[0]    
        
        ner_preds = list(map(int, (map(np.argmax, ner_logits[0]))))
        ner_labels = list(map(int, np.array(slot_labels)[0]))

    print(f"intent prediction: {intent_preds}")
    print(f"intent label     : {intent_labels}")
    print(f"ner prediction: {ner_preds}")
    print(f"ner lablel    : {ner_labels}")
    print()


intent prediction: 46
intent label     : 46
ner prediction: [0, 0, 0, 0, 0, 7, 7, 7, 7, 7, 7, 7]
ner lablel    : [111, 0, 0, 111, 0, 111, 7, 7, 7, 111, 0, 0]

intent prediction: 31
intent label     : 31
ner prediction: [0, 0, 0, 0, 0, 59, 59, 59, 59, 59, 59, 59, 59, 0]
ner lablel    : [111, 0, 0, 111, 0, 0, 111, 0, 111, 0, 111, 0, 0, 0]

intent prediction: 1
intent label     : 1
ner prediction: [0, 0, 0, 0, 0, 77, 77, 77, 77, 77, 0, 0, 0, 0]
ner lablel    : [111, 0, 111, 0, 111, 77, 77, 77, 77, 77, 111, 20, 20, 20]

intent prediction: 57
intent label     : 57
ner prediction: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 69, 69, 69, 69, 69, 69, 69, 69]
ner lablel    : [111, 0, 0, 111, 0, 111, 0, 111, 0, 0, 0, 111, 0, 111, 0, 111, 69, 69, 69, 69]

intent prediction: 10
intent label     : 7
ner prediction: [0, 0, 0, 0, 0, 0, 0, 0]
ner lablel    : [111, 0, 0, 0, 111, 0, 0, 0]

intent prediction: 53
intent label     : 1
ner prediction: [0, 0, 80, 0, 0, 0]
ner lablel    : [111, 22, 22, 22, 111, 24]



/var/folders/43/28bd6rrs7qlb2z8bs12rlsgc0000gn/T/ipykernel_34900/3279847629.py:18: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  ner_labels = list(map(int, np.array(slot_labels)[0]))
